In [ ]:
#Iniciando com a instalação da ADK
%pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72

In [ ]:
#Importação dos módulos genai e types
from google import genai
from google.genai import types

#Importação das classes Agent, Runner, InMemorySessionService e google_search
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

#Importação do módulo de datas, para trabalhar com datas
from datetime import date

#Importação do textwrap, para manipular quebra de linhas e formatar textos
import textwrap

#Importação das funções display e Markdown, para estilizações
from IPython.display import display, Markdown

#Importação da biblioteca request, para interação HTTP
import requests

#Importação do módulo warning, para controle de avisos e função pra ignorar avisos.
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Configura a API Key do Google Gemini

import os

from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Dicas --- #
##########################################

def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informações sobre treino para alunos de nível intermediário.",
        tools=[google_search],
        instruction="""
        Você é um assistente de pesquisas, que está auxiliando um aluno que possuem um pouco mais de experiência com exercícios físicos e prática de atividade física. Sua tarefa é usar a ferramenta de busca do google (google_search)para pesquisar por informações sobre treinos, atividades físicas, benefícios para a saúde através da atividade física e assuntos correlacionados ao mundo fitness, inclusive, dicas para o dia a dia saudável. Não seja prolixo, traga informações relevantes, de fácil compreensão e sempre traga informações com embassamento científico (não complique a cabeça do nosso aluno, embora de nível intermediário, ele é super prático no seu dia a dia). Se um conteúdo buscado não tiver muita qualidade, ou seja, pouco acesso, baixa relevância e sem embasamento científico, ignore e busque algo melhor. Essas informações devem ser mostradas em português brasileiro (se tiver algo de outro país, faça a tradução para o português do Brasil para que possa ser utilizado), de no máximo 3 meses antes da data de hoje. Se limite a trazer até 7 tópicos no conteúdo buscado.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [ ]:
################################################
# --- Agente 2: Planejador de treinos --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em atividade física, mundo fitness, educação física e personal trainner. Com base na lista de lançamentos mais recentes e relevantes buscados, você deve:	Usar a ferramenta de busca do Google (google_search) para criar um plano de treinamentos e prática de atividade física, sobre quais são os pontos mais relevantes que poderiamos abordar compor um plano de treinamento de atividades físicas. Você também pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar. Ao final, você irá escolher de três até sete práticas de atividades físicas, seus seus pontos mais relevantes, e um plano com esses tópicos a serem abordados em um planto de treinos que será desenvolvido posteriormente.
        """,
        description="Agente que planeja treinamentos",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Prescritor do Treinos --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um prescritor de treinos, especialista em mundo fitness e educação física. Você prescreve treinos para a empresa FitNerd, a maior rede de academias do Brasil. Utilize o material fornecido no plano de treino e os pontos mais relevantes fornecidos e, com base nisso, escreva um plano de treino que inclua a execução perfeita da atividade física e as melhores práticas a serem adotadas para este plano em um rascunho que será utilizado na academia FitNerd. O plano deve ser engajador, informativo, com linguagem simples e incluir de 3 até 7 exercícios.
            """,
        description="Agente redator de planos de treino para a academia FitNerd"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em academias, prática de atividades física, mundo fitness e educação física. Você está atendendo alunos do nível intermediário, logo possuem uma boa compreensão de termos de academia, estão seguindo com o seu cuidado do físico e saúde. Revise o rascunho do plano de treinamento da academia "FitNerd" verificando clareza, concisão, correção e tom. Se o plano de treino estiver bom, responda apenas 'O plano está ótimo e pronto para ser gerado!'. Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de plano de treinamentos para a FitNerd."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de apoio ao aluno iniciante da FitNerd com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer gerar o seu treino: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
	print("Você esqueceu de digitar o tópico!")
else:
	print (f"Maravilha! Vamos então vamos criar o seu plano: {topico}")

lancamentos_buscados = agente_buscador(topico, data_de_hoje)
print("\n--- Resultados do Agente 1 (Buscador) ---\n")
display(to_markdown(lancamentos_buscados))
print("----------------------------------------------------")

plano_de_post = agente_planejador(topico, lancamentos_buscados)
print("\n--- Resultado do Agente 2 (Planejador) ---\n")
display(to_markdown(plano_de_post))
print("----------------------------------------------------")

rascunho_de_post = agente_redator(topico, plano_de_post)
print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
display(to_markdown(rascunho_de_post))
print("--------------------------------------------------------------")

post_final = agente_revisor(topico, rascunho_de_post)
print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
display(to_markdown(post_final))
print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de apoio ao aluno iniciante da FitNerd com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer gerar o seu treino: Treino de peito
Maravilha! Vamos então vamos criar o seu plano de treinamento Treino de peito

--- Resultados do Agente 1 (Buscador) ---



> Olá! Para otimizar seu treino de peito, preparei algumas dicas e informações relevantes com base em pesquisas recentes. Veja só:
> 
>  
> Com certeza! Para te ajudar a montar um treino de peito eficiente, aqui estão algumas dicas valiosas e informações relevantes:
> 
>  1. **Exercícios Essenciais:**
>  *   **Supino Reto:** Clássico para trabalhar a parte central do peitoral.
>  *   **Supino Inclinado:** Ideal para a parte superior do peito (cabeça clavicular), contribuindo para uma estética mais completa.
>  *   **Supino Declinado:** Ótimo para a porção inferior do peitoral, ajudando a criar a impressão de um peitoral mais volumoso.
>  *   **Crucifixo:** Essencial para trabalhar o peitoral de forma isolada, permitindo uma maior amplitude e alongamento das fibras musculares.
>  *   **Voador (Peck Deck):** Outro exercício de isolamento que estabiliza o movimento, ótimo para quem tem dificuldade em sentir o peitoral trabalhando em exercícios como o supino.
>  *   **Flexões:** Um exercício fundamental que pode ser feito em qualquer lugar, ativando o peitoral de forma similar ao supino reto.
> 
>  2. **Dicas para Hipertrofia:**
>  *   **Amplitude Máxima e Controle:** Priorize a execução correta dos movimentos, garantindo que o peitoral seja ativado ao máximo em cada repetição.
>  *   **Cargas Adequadas:** Use cargas que permitam uma execução perfeita, com ajuda apenas para técnicas avançadas ou segurança.
>  *   **Frequência:** Treine peito de 1 a 2 vezes por semana, ajustando a frequência de acordo com sua recuperação e necessidades.
>  *   **Técnicas Avançadas:** Utilize técnicas como dropsets, supersets e pré-exaustão para intensificar o treino e promover maior estímulo muscular.
> 
>  3. **Priorize a Técnica:**
>  *   **Evite Abdução Escapular:** Não deixe os ombros avançarem durante os exercícios para evitar lesões e garantir que o foco permaneça no peitoral.
>  *   **Conexão Mente-Músculo:** Concentre-se em sentir o peitoral trabalhando durante cada repetição, maximizando a ativação muscular.
> 
>  4. **Variações e Ordem dos Exercícios:**
>  *   **Mude a Ordem:** Comece com exercícios diferentes do habitual para um novo estímulo. Por exemplo, inicie com o supino inclinado ou voador antes do supino reto.
>  *   **Use Halteres:** Trocar a barra por halteres pode aumentar a amplitude e o recrutamento muscular em alguns exercícios.
>  *   **Crossover com Polia:** Ajuste as polias em diferentes alturas (alta, média e baixa) para trabalhar todas as partes do peitoral.
> 
>  5. **Flexibilidade e Alongamento:**
>  *   **Alongamento:** Alongue o peitoral após o treino para melhorar a flexibilidade e recuperação muscular.
> 
>  6. **Importância do Treino de Peito:**
>  *   **Estabilização:** Melhora a estabilização da articulação da escápula e o fortalecimento do tronco.
>  *   **Equilíbrio:** Contribui para o desenvolvimento do volume muscular, aumentando a força e o equilíbrio corporal.
> 
>  7. **Exemplo de Treino:**
>  *   Supino reto com barra: 3 séries de 8-12 repetições
>  *   Supino inclinado com halteres: 3 séries de 8-12 repetições
>  *   Crucifixo reto: 3 séries de 10-15 repetições
>  *   Voador (Peck Deck): 3 séries de 10-15 repetições
>  *   Flexões: 3 séries até a falha
> 
>  Lembre-se sempre de consultar um profissional de educação física para personalizar o treino de acordo com suas necessidades e objetivos, garantindo a segurança e eficácia dos exercícios.
> 


----------------------------------------------------

--- Resultado do Agente 2 (Planejador) ---



> Com base nas informações fornecidas, podemos criar um plano de treinamento de peito abrangente, abordando os seguintes pontos-chave:
> 
>  **1. Exercícios Fundamentais e Variações:**
> 
>  *   **Supino Reto:** Variações com barra e halteres. Ênfase na técnica e amplitude.
>  *   **Supino Inclinado:** Variações com barra e halteres. Foco na porção superior do peito.
>  *   **Supino Declinado:** Para a porção inferior do peitoral.
>  *   **Crucifixo:** Reto, inclinado e declinado. Importância da amplitude e alongamento.
>  *   **Voador (Peck Deck):** Estabilização e isolamento do peitoral.
>  *   **Flexões:** Variações para diferentes níveis de dificuldade.
> 
>  **2. Técnicas de Intensificação:**
> 
>  *   **Dropsets:** Redução da carga para continuar o exercício até a falha.
>  *   **Supersets:** Combinação de dois exercícios sem descanso.
>  *   **Pré-exaustão:** Isolamento antes de exercícios compostos.
> 
>  **3. Ordem dos Exercícios e Variação:**
> 
>  *   **Priorização:** Alternar a ordem dos exercícios para estimular diferentes áreas do peitoral.
>  *   **Halteres vs. Barra:** Benefícios do uso de halteres para amplitude e recrutamento muscular.
>  *   **Crossover com Polia:** Ajuste das polias para trabalhar diferentes ângulos.
> 
>  **4. Foco na Técnica e Conexão Mente-Músculo:**
> 
>  *   **Amplitude Máxima:** Garantir a ativação completa do peitoral em cada repetição.
>  *   **Controle:** Evitar movimentos bruscos e manter o foco na contração muscular.
>  *   **Conexão Mente-Músculo:** Concentrar-se em sentir o peitoral trabalhando.
> 
>  **5. Flexibilidade e Recuperação:**
> 
>  *   **Alongamento:** Importância do alongamento pós-treino para flexibilidade e recuperação.
> 
>  **6. Frequência e Volume:**
> 
>  *   **Frequência:** Ajustar a frequência (1-2 vezes por semana) de acordo com a recuperação.
>  *   **Volume:** Variar o número de séries e repetições para evitar platôs.
> 
>  **7. Progressão e Personalização:**
> 
>  *   **Sobrecarga Progressiva:** Aumentar gradualmente a carga ou a dificuldade dos exercícios.
>  *   **Personalização:** Adaptar o treino às necessidades individuais e objetivos.
> 
>  **Plano de Treinamento Proposto:**
> 
>  1.  **Supino Inclinado com Halteres:** 3 séries de 8-12 repetições.
>  2.  **Supino Reto com Barra:** 3 séries de 8-12 repetições.
>  3.  **Crucifixo Reto:** 3 séries de 10-15 repetições.
>  4.  **Voador (Peck Deck):** 3 séries de 10-15 repetições (dropset na última série).
>  5.  **Crossover com Polia (alta):** 3 séries de 12-15 repetições.
>  6.  **Flexões:** 3 séries até a falha.
>  7.  **Alongamento do Peitoral:** 2-3 minutos.
> 
>  Este plano de treinamento pode ser adaptado e ajustado de acordo com o nível de experiência, objetivos e disponibilidade de cada indivíduo. É sempre recomendado consultar um profissional de educação física para garantir a segurança e eficácia do treino.
> 
>  Gostaria de ajuda para refinar o plano, com mais detalhes sobre progressão de cargas ou variações de exercícios?
> 


----------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> ## Rascunho do Treino de Peito FitNerd: Peitoral de Aço!
> 
> **Fala, Nerds!** Preparados para turbinar o peitoral e alcançar resultados incríveis? A FitNerd preparou um plano de treino completo, focado em técnica, intensidade e progressão para você conquistar o peito dos seus sonhos. 💪
> 
> **Importante:** Consulte sempre um profissional de educação física da FitNerd para adaptar o treino às suas necessidades e garantir a execução correta dos exercícios.
> 
> **Aquecimento (5 minutos):**
> 
> *   Cardio leve (esteira, bicicleta)
> *   Mobilidade articular (rotação de ombros, braços e punhos)
> *   2 séries de 15 repetições de flexões no joelho
> 
> **Treino (30-40 minutos):**
> 
> **1. Supino Inclinado com Halteres (3 séries de 8-12 repetições):**
> 
> *   **Por que?** Foca na porção superior do peitoral, dando aquele "upgrade" no visual.
> *   **Como fazer:** Deite-se em um banco inclinado (30-45 graus), segurando um halter em cada mão. Desça os halteres controladamente até a altura do peito e empurre-os de volta à posição inicial, contraindo o peitoral.
> *   **Dica FitNerd:** Mantenha os cotovelos levemente flexionados para evitar lesões e sinta o músculo trabalhar em cada repetição.
> 
> **2. Supino Reto com Barra (3 séries de 8-12 repetições):**
> 
> *   **Por que?** Exercício fundamental para construir massa muscular e força no peitoral.
> *   **Como fazer:** Deite-se em um banco reto, com a barra posicionada acima do peito. Segure a barra com as mãos um pouco mais afastadas que a largura dos ombros. Desça a barra controladamente até o peito e empurre-a de volta à posição inicial.
> *   **Dica FitNerd:** Peça ajuda a um parceiro para garantir a segurança, especialmente ao lidar com cargas mais altas.
> 
> **3. Crucifixo Reto (3 séries de 10-15 repetições):**
> 
> *   **Por que?** Alonga e define o peitoral, promovendo um maior recrutamento muscular.
> *   **Como fazer:** Deite-se em um banco reto, segurando um halter em cada mão. Abra os braços lateralmente, mantendo uma leve flexão nos cotovelos, até sentir um alongamento no peitoral. Retorne à posição inicial, contraindo o peitoral.
> *   **Dica FitNerd:** Concentre-se em sentir o alongamento e a contração do peitoral em cada repetição.
> 
> **4. Voador (Peck Deck) (3 séries de 10-15 repetições + Dropset na última série):**
> 
> *   **Por que?** Isola o peitoral, proporcionando um trabalho intenso e focado.
> *   **Como fazer:** Sente-se na máquina, ajuste a altura dos braços e segure as alças. Feche os braços à frente do corpo, contraindo o peitoral. Retorne à posição inicial controladamente.
> *   **Dica FitNerd:** Na última série, após atingir a falha muscular, reduza a carga e continue executando o exercício até a exaustão (dropset).
> 
> **5. Crossover com Polia (alta) (3 séries de 12-15 repetições):**
> 
> *   **Por que?** Permite trabalhar diferentes ângulos do peitoral, promovendo um desenvolvimento completo.
> *   **Como fazer:** Posicione-se no centro da máquina, segurando as polias altas. Incline o corpo levemente à frente e puxe as polias em direção ao centro do corpo, contraindo o peitoral. Retorne à posição inicial controladamente.
> *   **Dica FitNerd:** Ajuste a altura das polias para variar o ângulo de trabalho e atingir diferentes áreas do peitoral.
> 
> **6. Flexões (3 séries até a falha):**
> 
> *   **Por que?** Exercício funcional que trabalha o peitoral, ombros e tríceps, além de fortalecer o core.
> *   **Como fazer:** Apoie as mãos no chão, com os braços na largura dos ombros. Abaixe o corpo até o peito quase tocar o chão e empurre-o de volta à posição inicial.
> *   **Dica FitNerd:** Se as flexões tradicionais forem muito difíceis, apoie os joelhos no chão para facilitar o movimento.
> 
> **7. Alongamento do Peitoral (2-3 minutos):**
> 
> *   **Por que?** Melhora a flexibilidade, previne lesões e auxilia na recuperação muscular.
> *   **Como fazer:**
>     *   **Alongamento da porta:** Posicione-se em uma porta, segurando as laterais com as mãos. Incline o corpo à frente até sentir um alongamento no peitoral.
>     *   **Alongamento com as mãos atrás das costas:** Entrelace os dedos atrás das costas e estique os braços para trás, abrindo o peito.
> *   **Dica FitNerd:** Mantenha cada alongamento por 20-30 segundos, respirando profundamente.
> 
> **Progressão:**
> 
> *   **Carga:** Aumente a carga gradualmente, sempre que possível, mantendo a técnica correta.
> *   **Repetições:** Se conseguir realizar mais de 12 repetições com facilidade, aumente a carga.
> *   **Variação:** Varie os exercícios a cada 4-6 semanas para evitar platôs e manter o estímulo muscular.
> *   **Técnicas de Intensificação:** Utilize técnicas como dropsets e supersets para aumentar a intensidade do treino.
> 
> **Recuperação:**
> 
> *   **Descanso:** Descanse adequadamente entre os treinos (48-72 horas) para permitir a recuperação muscular.
> *   **Alimentação:** Consuma proteínas e carboidratos de qualidade para fornecer energia e nutrientes para a recuperação muscular.
> *   **Sono:** Durma de 7 a 8 horas por noite para otimizar a recuperação e o crescimento muscular.
> 
> **Observações:**
> 
> *   Mantenha a postura correta durante todos os exercícios.
> *   Controle a respiração, expirando na fase concêntrica (contração) e inspirando na fase excêntrica (relaxamento).
> *   Concentre-se em sentir o músculo trabalhando em cada repetição.
> *   Não tenha pressa, execute os exercícios com calma e controle.
> *   Divirta-se e desafie-se a cada treino!
> 
> **#FitNerd #TreinoDePeito #PeitoralDeAço #VemPraFitNerd**


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O plano está ótimo e pronto para ser gerado!


--------------------------------------------------------------
